In [2]:
import sys
sys.path.append('gplearn')

%matplotlib inline
from gplearn.genetic import SymbolicRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import graphviz


# Ground truth
x0 = np.arange(-1, 1, .1)
x1 = np.arange(-1, 1, .1)
x0, x1 = np.meshgrid(x0, x1)
y_truth = x0**2 - x1**2 + x1 - 1
rng = check_random_state(0)


# Training samples
X_train = rng.uniform(-1, 1, 1000).reshape(10, 50, 2)
y_train = X_train[:, :, 0]**2 - X_train[:, :, 1]**2 + X_train[:, :, 1] - 1


# Testing samples
X_test = rng.uniform(-1, 1, 1000).reshape(10, 50, 2)
y_test = X_test[:, :, 0]**2 - X_test[:, :, 1]**2 + X_test[:, :, 1] - 1
est_gp = SymbolicRegressor(population_size=5000,
                           generations=20, stopping_criteria=0.01,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0,
                           n_jobs=10)
est_gp.fit(X_train, y_train)
print(est_gp._program)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    38.13      4.03172e+06        9         0.340105          0.32787     10.99s
   1     9.54          2.69158        5         0.333636         0.372242     53.73s
   2     7.15          108.894       11         0.268337         0.282004     57.73s
   3     4.93          1.46701       11         0.247765         0.241173     52.16s
   4     3.58         0.850937        7         0.238789         0.319314     50.34s
   5     4.19          1.94188       13         0.172865         0.172865     51.30s
   6     5.69          1.32474       13         0.134246         0.134246     45.56s
   7     7.40          2.19815       13         0.134246         0.134246     38.27s
   8     7.73          1.31063       13        0.0570983        0.0570983  